# Deliverable 1
Student name = Aditya Bhavsar


In [ ]:
pip install requests openai newspaper3k google-generativeai sentence-transformers


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install newspaper3k

In [ ]:
!pip install lxml_html_clean lxml newspaper3k --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=9c97e43c5e00a7c88b135a77ee2d6dc70db632db81d5d583c50b42f2614fe194
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=fe879fd82a1fc9d12b36e6265f03682b0a28b297d0512694db5762acd45962ba
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [ ]:
!pip install google-search-results

In [ ]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import requests
import tldextract
from datetime import datetime
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import openai  # For AI-based fact-checking

# === SERPAPI & OpenAI API Keys ===
SERP_API_KEY = "Your api key" # Replace with your OpenAI API Key
OPENAI_API_KEY = "Your api key"  # Replace with your OpenAI API Key

openai.api_key = OPENAI_API_KEY

# === 1st metric - Domain Trust ===
def get_domain_trust(url):
    """Estimates domain trust using search ranking, WHOIS data, and Tranco list."""
    domain = tldextract.extract(url).registered_domain

    # === Check Tranco List ===
    try:
        tranco_response = requests.get("https://tranco-list.eu/top-1m.csv").text
        if domain in tranco_response:
            tranco_rank = tranco_response.split(domain)[0].strip().split("\n")[-1]
            tranco_score = max(100 - (int(tranco_rank) / 10000), 50)
        else:
            tranco_score = 40
    except:
        tranco_score = 40

    # === WHOIS Lookup for Domain Age ===
    try:
        whois_response = requests.get(f"https://api.ip2whois.com/v2?key=demo&domain={domain}").json()
        creation_date = whois_response.get("created_date", "2000-01-01")
        domain_age = (datetime.now() - datetime.strptime(creation_date, "%Y-%m-%d")).days // 365
        age_score = min(domain_age * 5, 100)
    except:
        age_score = 50

    # === Extract Backlink Score (Using SerpAPI) ===
    try:
        search = requests.get(f"https://serpapi.com/search.json?q={domain}&api_key={SERP_API_KEY}").json()
        backlink_count = len(search.get("organic_results", []))
        backlink_score = min(backlink_count * 10, 100)
    except:
        backlink_score = 50

    # === Final Domain Trust Calculation ===
    domain_trust = (0.4 * tranco_score) + (0.3 * age_score) + (0.3 * backlink_score)
    return round(domain_trust, 2)

# === 2nd metric - AI Based fact check score ===
import requests
from sentence_transformers import SentenceTransformer, util

def get_fact_check_score(text):
    """Evaluates the factual accuracy of a given claim using multiple layers of verification."""

    # === Step 1: Verify with Search Engine Results ===
    try:
        serpapi_key = "78d578eb49c83a63f5945da59e8ae9e37f865cc14da2c714ccb9e990a195d787"
        params = {
            "q": f"fact check {text}",
            "engine": "google",
            "api_key": serpapi_key
        }
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()

        # Check for fact-checking websites in top results
        trusted_sources = ["snopes.com", "politifact.com", "factcheck.org", "bbc.com", "reuters.com"]
        source_mentions = sum(1 for result in data.get("organic_results", []) if any(domain in result.get("link", "") for domain in trusted_sources))

        fact_check_score = min(source_mentions * 20, 100)  # Normalize (max 5 sources = 100)
    except Exception:
        fact_check_score = 50  # Default neutral score

    # === Step 2: Cross-Check with Wikipedia ===
    try:
        wiki_response = requests.get(f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={text}")
        wiki_data = wiki_response.json()
        wiki_matches = len(wiki_data.get("query", {}).get("search", []))

        fact_check_score += min(wiki_matches * 10, 30)  # Add extra points if Wikipedia has related articles
    except Exception:
        pass  # Ignore errors, keep previous score

    # === Step 3: NLP Semantic Similarity ===
    try:
        model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        trusted_texts = " ".join([result["title"] + result["snippet"] for result in data.get("organic_results", []) if "title" in result and "snippet" in result])

        similarity_score = util.pytorch_cos_sim(model.encode(text), model.encode(trusted_texts)).item() * 100
        fact_check_score += min(similarity_score / 2, 30)  # Weight this factor to prevent over-scaling
    except Exception:
        pass  # Ignore errors, keep previous score

    # === Final Normalization ===
    return max(0, min(fact_check_score, 100))  # Ensure within range

# === 3rd metric - Bias Score ===
def get_bias_score(text, domain):
    """Calculates bias score based on sentiment analysis and media bias ratings."""
    sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
    sentiment_result = sentiment_pipeline(text[:512])[0]

    # Map sentiment to bias score
    sentiment_bias = {
        "1 star": 30,  # Strongly Negative (Potentially biased)
        "2 stars": 50,  # Slightly Negative (Neutral to biased)
        "3 stars": 70,  # Neutral
        "4 stars": 80,  # Slightly Positive (Neutral to trusted)
        "5 stars": 100  # Strongly Positive (Trusted)
    }
    bias_score = sentiment_bias.get(sentiment_result["label"], 50)

    # === Fetch Bias Rating from AllSides API ===
    try:
        bias_response = requests.get(f"https://api.allsides.com/bias/{domain}").json()
        media_bias = bias_response.get("bias", "center")
        bias_adjustment = {
            "left": -20,
            "lean left": -10,
            "center": 0,
            "lean right": 10,
            "right": 20
        }.get(media_bias.lower(), 0)
        bias_score = max(0, min(bias_score + bias_adjustment, 100))
    except:
        pass  # If API fails, use sentiment score only

    return round(bias_score, 2)

# === 4th metric - Content Relevance Score ===
def compute_content_relevance(user_query, page_text):
    """Computes content relevance using semantic similarity."""
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    similarity_score = util.pytorch_cos_sim(model.encode(user_query), model.encode(page_text)).item() * 100
    return round(similarity_score, 2)

# === 5th metric - Citation Score ===
def check_google_scholar(url):
    """
    Computes Citation Score by:
    1) Checking backlinks using Google Search (SerpAPI).
    2) Searching for academic mentions (PDFs, research papers).
    3) Normalizing results to a 0-100 scale.
    """
    try:
        # === Step 1: Count Backlinks from Google Search ===
        backlink_params = {
            "q": f"link:{url}",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        backlink_response = requests.get("https://serpapi.com/search", params=backlink_params).json()
        backlink_count = len(backlink_response.get("organic_results", []))

        # === Step 2: Count Academic Mentions (PDFs & Research Papers) ===
        academic_params = {
            "q": f"\"{url}\" filetype:pdf OR site:researchgate.net OR site:arxiv.org OR site:semanticscholar.org",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        academic_response = requests.get("https://serpapi.com/search", params=academic_params).json()
        academic_count = len(academic_response.get("organic_results", []))

        # === Step 3: Normalize Score to 0-100 ===
        citation_score = min((backlink_count * 5) + (academic_count * 15), 100)  # Adjust weights if needed

        return round(citation_score, 2)

    except Exception as e:
        print(f"Error fetching citation data: {e}")
        return 0  # Default if API fails

# === Final Step - Final score evaluation method ===
def rate_url_validity(user_query, url):
    """
    Evaluates the validity of a given URL by computing multiple credibility metrics.
    Returns a final credibility score (0-100).
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        page_text = " ".join([p.text for p in soup.find_all("p")])  # Extract paragraph text
    except Exception as e:
        return {"error": f"Failed to fetch content: {str(e)}"}

    domain = tldextract.extract(url).registered_domain

    # Compute all credibility scores
    domain_trust = get_domain_trust(url)
    content_relevance = compute_content_relevance(user_query, page_text)
    fact_check_score = get_fact_check_score(page_text)
    bias_score = get_bias_score(page_text, domain)
    citation_score = check_google_scholar(url)

    # === Compute Final Validity Score ===
    final_score = (
        (0.15 * domain_trust) +
        (0.40 * content_relevance) +
        (0.10 * fact_check_score) +
        (0.20 * bias_score) +
        (0.15 * citation_score)
    )

    return {
        "Domain Trust": domain_trust,
        "Content Relevance": content_relevance,
        "Fact-Check Score": fact_check_score,
        "Bias Score": bias_score,
        "Citation Score": citation_score,
        "Final Validity Score": round(final_score, 2)
    }


In [ ]:
# === TESTING ===
test_urls = [
    "https://www.bbc.com/news",
    "https://www.nasa.gov",
    "https://www.some-random-blog.com",
    "https://www.factcheck.org"
]

for url in test_urls:
    print(f"\nResults for {url}:")
    print(rate_url_validity("climate change effects", url))


Results for https://www.bbc.com/news:


Device set to use cpu


{'Domain Trust': 61.0, 'Content Relevance': 17.01, 'Fact-Check Score': 50, 'Bias Score': 50, 'Citation Score': 100, 'Final Validity Score': 49.25}

Results for https://www.nasa.gov:


Device set to use cpu


{'Domain Trust': 58.0, 'Content Relevance': 16.92, 'Fact-Check Score': 30, 'Bias Score': 80, 'Citation Score': 100, 'Final Validity Score': 45.78}

Results for https://www.some-random-blog.com:
{'error': 'Failed to fetch content: 406 Client Error: Not Acceptable for url: https://www.some-random-blog.com/'}

Results for https://www.factcheck.org:


Device set to use cpu


{'Domain Trust': 64.0, 'Content Relevance': 14.16, 'Fact-Check Score': 100, 'Bias Score': 30, 'Citation Score': 80, 'Final Validity Score': 58.85}


In [ ]:
url = "https://www.nvidia.com/en-us/geforce/graphics-cards/50-series/rtx-5090/"
user_query = "rtx 5090 nvidia"
rate_url_validity(user_query, url)

Device set to use cpu


{'Domain Trust': 64.0,
 'Content Relevance': 61.11,
 'Fact-Check Score': 30,
 'Bias Score': 100,
 'Citation Score': 100,
 'Final Validity Score': 61.93}